In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import numpy as np

# User agent
HEADERS = {'User-Agent': 'Mozilla/5.0'}
    
# Webpage URL of poznan properties and base URL of the page
BASE_URL = 'https://www.otodom.pl'
MAIN_URL = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/wielkopolskie/poznan/poznan/poznan?viewType=listing%2Fpage%3D3%2Fpage%3D2&'


In [13]:
response = requests.get(MAIN_URL, headers=HEADERS)
if response.status_code == 200:
    print("Successfully fetched the page")
else:
    print(f"Failed to fetch page {print(f"Failed to fetch the page with status code {response.status_code}")}")


Successfully fetched the page


In [39]:
soup = BeautifulSoup(response.text, "html.parser")
links = soup.find_all('a', attrs={'data-cy' : 'listing-item-link'})
link = links[10].get('href')
product = BASE_URL + link
new_response = requests.get(product, headers=HEADERS)
new_soup = BeautifulSoup(new_response.text, "html.parser")
id = new_soup.find("p", attrs={"class": "e1aeqjrs2 css-htq2ld"}).text.replace('ID: ', '')
price = new_soup.find('strong', attrs={"class" : "css-1o51x5a e1k1vyr21"}).text.replace(" zł", "").replace(" ", "")
sqr_meters = new_soup.find('div', attrs={"class" : "css-1ftqasz"}).text
num_rooms = new_soup('div', attrs={"class" : "css-1ftqasz"})[1].text.replace(" pokój", "")
price_square_ratio = new_soup.find('div', attrs={"class" : "css-z3xj2a e1k1vyr25"}).text.replace(' zł/m²', '').replace(' ', '')
location = new_soup.find('a', attrs={"class" : "css-1jjm9oe e42rcgs1"}).text
print(id)
print(price)
print(sqr_meters)
print(num_rooms)
print(price_square_ratio)
print(location)

66542152
875000
59m²
3 pokoje
14831
ul. Smardzewska 7, Raszyn, Grunwald, Poznań, wielkopolskie


In [19]:
info = new_soup('p', attrs = {"class" : "eows69w2 css-1airkmu"})
info_dict = {}

for i in range(0, len(info)-1, 2):
    key = info[i].text.strip().replace("<!-- -->:", "").strip().upper()
    value = info[i+1].text.strip()
    if not value:
        value = 'null'
    info_dict[key] = value
    
print(info_dict)
print(len(info_dict))

{'OGRZEWANIE:': 'gazowe', 'PIĘTRO:': 'parter/1', 'CZYNSZ:': 'Stan wykończenia:', 'DO ZAMIESZKANIA': 'Rynek:', 'WTÓRNY': 'Forma własności:', 'PEŁNA WŁASNOŚĆ': 'Dostępne od:', 'TYP OGŁOSZENIODAWCY:': 'biuro nieruchomości', 'INFORMACJE DODATKOWE:': 'balkon ogródek garaż/miejsce parkingowe', 'WINDA:': 'nie', 'RODZAJ ZABUDOWY:': 'apartamentowiec', 'MATERIAŁ BUDYNKU:': 'cegła', 'OKNA:': 'plastikowe', 'CERTYFIKAT ENERGETYCZNY:': 'A+', 'BEZPIECZEŃSTWO:': 'teren zamknięty', 'WYPOSAŻENIE:': 'zmywarka lodówka meble piekarnik kuchenka', 'ZABEZPIECZENIA:': 'system alarmowy drzwi / okna antywłamaniowe domofon / wideofon', 'MEDIA:': 'telewizja kablowa internet telefon'}
17


In [40]:
df = pd.read_csv(r'/Users/franekbiskup/Desktop/Portfolio/PolandPropertyScraper/project/data/poznan_properties.csv')
df.dtypes

property_id        int64
price            float64
square           float64
price_per_sqm    float64
rooms              int64
date              object
url               object
street            object
district_1        object
district_2        object
city              object
state             object
dtype: object

In [3]:
def get_links(main_url= MAIN_URL, base_url= BASE_URL):
    
    # HTTP request
    response = requests.get(main_url, headers=HEADERS)
    if response.status_code == 200:
        print("Successfully fetched the page")
    else:
        print(f"Failed to fetch page {print(f"Failed to fetch the page with status code {response.status_code}")}")
    
    # BS4 object containing all data
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Fetch all links as list of Tag Objects
    hrefs_list = []
    links = soup.find_all('a', attrs={'data-cy' : 'listing-item-link'})
    
    # Loop for extracting links from Tag Objects
    for link in links:
        hrefs_list.append(link['href'])
        
    links_list = [urljoin(base_url, href) for href in hrefs_list]

    return links_list

In [4]:
get_links()

Successfully fetched the page


['https://www.otodom.pl/pl/oferta/wilda80-m-4-pokoje-2-balkony-zamkniete-osiedle-ID4uOMS',
 'https://www.otodom.pl/pl/oferta/duze-pokoje-cale-wyposazenie-w-cenie-piwnica-ID4wfpk',
 'https://www.otodom.pl/pl/oferta/3-pok-marcelinska-ataner-ID4uOcg',
 'https://www.otodom.pl/pl/oferta/katowicka-ataner-polanka-2-dwa-mieszkania-roi-6-ID4hnR5',
 'https://www.otodom.pl/pl/oferta/wilda80-m-4-pokoje-2-balkony-zamkniete-osiedle-ID4uOMS',
 'https://www.otodom.pl/pl/oferta/komfortowy-apt-blisko-parku-solackiego-ochrona-24h-ID4w81l',
 'https://www.otodom.pl/pl/oferta/mieszkanie-na-jezycach-bez-posrednikow-ID4ubfE',
 'https://www.otodom.pl/pl/oferta/mieszkanie-na-sprzedaz-garaz-piwnica-red-park-ID4u9mw',
 'https://www.otodom.pl/pl/oferta/piekny-apartament-z-garderoba-0pcc-ID4wcEv',
 'https://www.otodom.pl/pl/oferta/ekskluzywny-apartament-5-min-pieszo-do-lasku-marc-ID4wjZV',
 'https://www.otodom.pl/pl/oferta/4-pokojowe-mieszkanie-81m2-loggia-bez-prowizji-ID4un9X',
 'https://www.otodom.pl/pl/oferta/br

In [5]:
# Function to extract IDs
def get_ids(soup):
    try:
        id = soup.find("p", attrs={"class": "e1aeqjrs2 css-htq2ld"}).text.replace('ID: ', '')
        return id
    except AttributeError:
        return None


# Function to extract Price
def get_price(soup):
    try:
        price = soup.find('strong', attrs={"class" : "css-1o51x5a e1k1vyr21"}).text.replace(" zł", "").replace(" ", "")
        return price
    except AttributeError:
        return None


# Function to extract Square Meters
def get_sqm(soup):
    try:
        sqr_meters = soup('div', attrs={"class" : "css-1ftqasz"})[0].text
        return sqr_meters
    except AttributeError:
        return None
    
    
# Function to extract Number of Rooms
def get_numer_of_rooms(soup):
    try:
        num_rooms = soup('div', attrs={"class" : "css-1ftqasz"})[1].text.replace(" pokój", "").replace(" pokoje", "")
        return num_rooms
    except AttributeError:
        return None
    
    
# Function to extract Price per Square Meters
def get_price_per_sqm(soup):
    try:
        price_square_ratio = soup('div', attrs={"class" : "css-z3xj2a e1k1vyr25"})[0].text.replace(' zł/m²', '').replace(' ', '')
        return price_square_ratio
    except AttributeError:
        return None
    

# Function to extract Location
def get_location(soup):
    try:
        location = soup('a', attrs={"class" : "css-1jjm9oe e42rcgs1"})[0].text
        return price
    except AttributeError:
        return None
    